In [1]:
import numpy as np
import tensorflow as tf
import random
import math
import h5py
import gc
import sys
import pickle
import PIL
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
def acc(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])

In [3]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

In [4]:
def plot_img(im, l='label'):
    if len(im.shape) >= 3:
        im = im[:,:,0]
    plt.imshow(im)
    plt.title(l)
    plt.show()

In [5]:
def prep_image(filename):
    img = Image.open('test_images/' + filename + '.jpg')
    img = img.resize((96,32), PIL.Image.ANTIALIAS)
    img = np.asarray(img, dtype="int32")
    img = np.dot(img, [[0.2989],[0.5870],[0.1140]])
    return img

In [6]:
train_images = ['0016','0123','206','212','214','312','468','470','597','720','907','950','980','6124','9765','0','1','01','0016','00016']
label_data = np.array([[4,10,10,1,6,0],[4,10,1,2,3,0],[3,2,10,6,0,0],[3,2,1,2,0,0],[3,2,1,4,0,0],[3,3,1,2,0,0],[3,4,6,8,0,0],[3,4,7,10,0,0],[3,5,9,7,0,0],[3,7,2,10,0,0],[3,9,10,7,0,0],[3,9,5,10,0,0],[3,9,8,10,0,0],[4,6,2,1,4,0],[4,9,7,6,5,0],[1,10,0,0,0,0],[1,1,0,0,0,0],[2,10,1,0,0,0],[4,10,10,1,6,0],[5,10,10,10,1,6]])

In [12]:
len(train_images)
label_data.shape

(20, 6)

In [7]:
image_data = np.ndarray([len(train_images),32,96,1])

for i in range(len(train_images)):
    image_name = train_images[i]
    image_data[i,:,:,:] = prep_image(image_name)[:,:,:]

In [13]:
image_data.shape

(20, 32, 96, 1)

In [8]:
mnist_model = "saved_models/mnist/CNN_SVHN_Mnist.ckpt"
box_model = "saved_models/box/CNN_SVHN_Box.ckpt"

In [9]:
graph_svhn = tf.Graph()

with graph_svhn.as_default():
    HEIGHT = 32
    WIDTH = 32*3

    X = tf.placeholder(tf.float32, [None, HEIGHT, WIDTH, 1])
    Y_ = tf.placeholder(tf.int32, [None, 6])
    
    # Learning Rate - alpha
    alpha = tf.placeholder(tf.float32)
    
    # Dropout Probablity
    pkeep = tf.placeholder(tf.float32)
    
    # 6 Layers and their no of neurons
    # 3 Convolutional Layers and a fully connected layer
    K = 12     # First Conv Layer with depth 12
    L = 24     # Second Conv Layer with depth 24
    M = 36    # Third Conv layer with depth 36
    N = 300   # Fourth Fully Connected layer with 300 neurons
    P = 200   # Fifth Fully Connected layer with 200 neurons
    # Last one will be softmax layer with 10 output channels
    
    W1 = tf.Variable(tf.truncated_normal([6, 6, 1, K], stddev=0.1), name="W1")    # 6x6 patch, 1 input channel, K output channels
    B1 = tf.Variable(tf.constant(0.1, tf.float32, [K]), name="B1")
    
    W2 = tf.Variable(tf.truncated_normal([5, 5, K, L], stddev=0.1), name="W2")
    B2 = tf.Variable(tf.constant(0.1, tf.float32, [L]), name="B2")
    
    W3 = tf.Variable(tf.truncated_normal([4, 4, L, M], stddev=0.1), name="W3")
    B3 = tf.Variable(tf.constant(0.1, tf.float32, [M]), name="B3")
    
    W5_1 = tf.Variable(tf.truncated_normal([P, 11], stddev=0.1), name="W5_1")
    B5_1 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_1")
    
    W5_2 = tf.Variable(tf.truncated_normal([P, 11], stddev=0.1), name="W5_2")
    B5_2 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_2")
    
    W5_3 = tf.Variable(tf.truncated_normal([P, 11], stddev=0.1), name="W5_3")
    B5_3 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_3")
    
    W5_4 = tf.Variable(tf.truncated_normal([P, 11], stddev=0.1), name="W5_4")
    B5_4 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_4")
    
    W5_5 = tf.Variable(tf.truncated_normal([P, 11], stddev=0.1), name="W5_5")
    B5_5 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_5")
    
    # Model
    stride = 1  # output is 32x96
    Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)
    
    stride = 2  # output is 16x48
    Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)
    
    stride = 2  # output is 8x24
    Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)

    # reshape the output from the third convolution for the fully connected layer
    shape = Y3.get_shape().as_list()
    YY = tf.reshape(Y3, shape=[-1, shape[1] * shape[2] * shape[3]])
    
    W4 = tf.Variable(tf.truncated_normal([shape[1] * shape[2] * shape[3], N], stddev=0.1), name="W4")
    B4 = tf.Variable(tf.constant(0.1, tf.float32, [N]), name="B4")
    
    W5 = tf.Variable(tf.truncated_normal([N, P], stddev=0.1), name="W5")
    B5 = tf.Variable(tf.constant(0.1, tf.float32, [P]), name="B5")

    Y4 = tf.nn.relu(tf.matmul(YY, W4) + B4)
    Y5 = tf.nn.relu(tf.matmul(Y4, W5) + B5)
    
    Y_F = tf.nn.dropout(Y5, pkeep)
    
    Ylogits_1 = tf.matmul(Y_F, W5_1) + B5_1
    Ylogits_2 = tf.matmul(Y_F, W5_2) + B5_2
    Ylogits_3 = tf.matmul(Y_F, W5_3) + B5_3
    Ylogits_4 = tf.matmul(Y_F, W5_4) + B5_4
    Ylogits_5 = tf.matmul(Y_F, W5_5) + B5_5   
    ## ('Ylogits_1 shape : ', [None, 11])
    
    Y_1 = tf.nn.softmax(Ylogits_1)
    Y_2 = tf.nn.softmax(Ylogits_2)
    Y_3 = tf.nn.softmax(Ylogits_3)
    Y_4 = tf.nn.softmax(Ylogits_4)
    Y_5 = tf.nn.softmax(Ylogits_5)
   
    cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_1, Y_[:,1])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_2, Y_[:,2])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_3, Y_[:,3])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_4, Y_[:,4])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_5, Y_[:,5]))

    train_prediction = tf.pack([Y_1, Y_2, Y_3, Y_4, Y_5])
    
    train_step = tf.train.AdamOptimizer(alpha).minimize(cross_entropy)
    
    W_s = tf.pack([tf.reduce_max(tf.abs(W1)),tf.reduce_max(tf.abs(W2)),tf.reduce_max(tf.abs(W3))\
                   ,tf.reduce_max(tf.abs(W4)),tf.reduce_max(tf.abs(W5))])
    b_s = tf.pack([tf.reduce_max(tf.abs(B1)),tf.reduce_max(tf.abs(B2)),tf.reduce_max(tf.abs(B3))\
                   ,tf.reduce_max(tf.abs(B4)),tf.reduce_max(tf.abs(B5))])
    
    model_saver = tf.train.Saver()

In [14]:
##### Test with MNIST - Trained Model on Real World Data

In [16]:
with tf.Session(graph=graph_svhn) as session: 
    model_saver.restore(session, mnist_model)
    print('Initialized')
 
    _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : image_data, Y_ : label_data, pkeep : 1.0, alpha : 0.002})
    accuracy = acc(predictions, label_data[:,1:6])

    print('Valid accuracy: ', accuracy) 

Initialized
Valid accuracy:  10.0
